In [14]:
import requests
import json
import pandas as pd

url = "https://howtheyvote.eu/api/votes/"

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

result = json.loads(response.text)

In [15]:
import requests
import json
import pandas as pd
import itertools

all_votes_list = []
i = 1
has_next = True
while has_next is True:

  url = "https://howtheyvote.eu/api/votes"

  querystring = {"page":str(i),"page_size":"200"}

  headers = {"Accept": "application/json"}

  response = requests.get(url, headers=headers, params=querystring)
  res = json.loads(response.text)

  all_votes_list.append(res["results"])
  has_next = res["has_next"]
  i+=1


all_votes_list_merged = list(itertools.chain(*all_votes_list))

In [16]:
all_labels = []
for item in all_votes_list_merged:
  if item['eurovoc_concepts'] != []:
    dicts = item['eurovoc_concepts']
    labels = [it['label'] for it in dicts]
    for label in labels:
      if label not in all_labels:
        all_labels.append(label)
all_labels.sort()

In [17]:
all_labels

['20 TRADE',
 '5G',
 '68 INDUSTRY',
 'ACP countries',
 'ACP-EU relationship',
 'ASEAN',
 'Antarctic Ocean',
 'Aragon',
 'Atlantic Ocean',
 'Attica',
 'Basque Country',
 'Black Sea',
 'Body of European Regulators for Electronic Communications',
 'CEPOL',
 'Catalonia',
 'Cedefop',
 'Cenelec',
 'Community acquis',
 'Community certification',
 'Community loan',
 'Community support framework',
 'Council of the European Union',
 'Court of Justice (EU)',
 'Court of Justice of the European Union',
 'EAEC',
 'EAFRD',
 'EAGF',
 'EAGGF',
 'EC conformity marking',
 'EDF',
 'EMS exchange-rate mechanism',
 'ETSI',
 'EU Emissions Trading Scheme',
 'EU Member State',
 'EU Official Journal',
 'EU action',
 'EU aid',
 'EU banking union',
 'EU body',
 'EU budget',
 'EU candidate countries',
 'EU competition policy',
 'EU control',
 'EU customs procedure',
 'EU emission allowance',
 'EU employment policy',
 'EU energy policy',
 'EU environmental policy',
 'EU expenditure',
 'EU financial instrument',
 'EU

In [55]:
votes_themes = {}
theme = "sustainable product"
votes_themes['eurovoc_concepts'] = theme
votes_themes["ids"] = []
for vote in all_votes_list_merged:
  eurovoc_ids = [item['label'] for item in vote["eurovoc_concepts"]]
  if theme in eurovoc_ids :
    votes_themes["ids"].append(vote["id"])

In [56]:
votes_themes["ids"]

['168504',
 '168424',
 '168481',
 '166226',
 '164532',
 '163059',
 '160802',
 '160139',
 '156936',
 '155533',
 '155050']

In [57]:
import requests
selected_votes = []
for vote_id in votes_themes["ids"]:
  url = f"https://howtheyvote.eu/api/votes/{vote_id}"

  headers = {"Accept": "application/json"}

  response = requests.get(url, headers=headers)
  selected_votes.append(json.loads(response.text))

In [59]:
votes = []
mb_ids = [197533,131580,197534,197694,135511,97236]
for vote in selected_votes:
  for member in vote["member_votes"]:
    if member['member']['id'] in mb_ids:
      to_store = {}
      to_store["member_id"] = member['member']['id']
      to_store["first_name"] =  member['member']['first_name']
      to_store["last_name"] =  member['member']['last_name']
      to_store["group"] =  member['member']['group']['label']
      to_store["photo_url"] =  member['member']['photo_url']
      to_store["position"] =  member['position']
      to_store["vote_id"] = vote["id"]
      to_store["display_title"] = vote["display_title"]
      to_store["timestamp"] = vote["timestamp"]
      to_store["facts"] = vote["facts"]
      to_store["sources"] = vote["sources"]
      votes.append(to_store)


In [60]:
df = pd.DataFrame.from_dict(votes)
df

,member_id,first_name,last_name,group,photo_url,position,vote_id,display_title,timestamp,facts,sources
0,197533,Manon,AUBRY,The Left in the European Parliament – GUE/NGL,/api/static/members/197533.jpg,FOR,168504,Packaging and packaging waste,2024-04-24T12:36:30,<ul><li>Measures cover full life cycle of pack...,[{'url': 'https://www.europarl.europa.eu/doceo...
1,131580,Jordan,BARDELLA,Identity and Democracy,/api/static/members/131580.jpg,AGAINST,168504,Packaging and packaging waste,2024-04-24T12:36:30,<ul><li>Measures cover full life cycle of pack...,[{'url': 'https://www.europarl.europa.eu/doceo...
2,197534,François-Xavier,BELLAMY,European People’s Party,/api/static/members/197534.jpg,AGAINST,168504,Packaging and packaging waste,2024-04-24T12:36:30,<ul><li>Measures cover full life cycle of pack...,[{'url': 'https://www.europarl.europa.eu/doceo...
3,197694,Raphaël,GLUCKSMANN,Progressive Alliance of Socialists and Democrats,/api/static/members/197694.jpg,FOR,168504,Packaging and packaging waste,2024-04-24T12:36:30,<ul><li>Measures cover full life cycle of pack...,[{'url': 'https://www.europarl.europa.eu/doceo...
4,135511,Valérie,HAYER,Renew Europe,/api/static/members/135511.jpg,FOR,168504,Packaging and packaging waste,2024-04-24T12:36:30,<ul><li>Measures cover full life cycle of pack...,[{'url': 'https://www.europarl.europa.eu/doceo...
...,...,...,...,...,...,...,...,...,...,...,...
61,131580,Jordan,BARDELLA,Identity and Democracy,/api/static/members/131580.jpg,FOR,155050,Empowering consumers for the green transition,2023-05-11T12:10:38,"<ul><li>General, unsubstantiated environmental...",[{'url': 'https://www.europarl.europa.eu/news/...
62,197534,François-Xavier,BELLAMY,European People’s Party,/api/static/members/197534.jpg,FOR,155050,Empowering consumers for the green transition,2023-05-11T12:10:38,"<ul><li>General, unsubstantiated environmental...",[{'url': 'https://www.europarl.europa.eu/news/...
63,197694,Raphaël,GLUCKSMANN,Progressive Alliance of Socialists and Democrats,/api/static/members/197694.jpg,FOR,155050,Empowering consumers for the green transition,2023-05-11T12:10:38,"<ul><li>General, unsubstantiated environmental...",[{'url': 'https://www.europarl.europa.eu/news/...
64,135511,Valérie,HAYER,Renew Europe,/api/static/members/135511.jpg,FOR,155050,Empowering consumers for the green transition,2023-05-11T12:10:38,"<ul><li>General, unsubstantiated environmental...",[{'url': 'https://www.europarl.europa.eu/news/...


In [121]:
for vote_id in list(df['vote_id'].unique()):
    vote_df = df[df['vote_id']==vote_id]
    title = vote_df["display_title"].unique()[0]
    print(title)
    for member in list(vote_df["member_id"]):
        mb_df = vote_df[vote_df["member_id"]==member]

Packaging and packaging waste
Common rules promoting the repair of goods
Ecodesign for Sustainable Products Regulation
Substantiation and communication of explicit environmental claims (Green Claims Directive)
Geographical Indications for wine, spirit drinks and agricultural products
Empowering consumers for the green transition
Packaging and packaging waste
Common rules promoting the repair of goods
Ecodesign for Sustainable Products Regulation
Geographical Indications for wine, spirit drinks and agricultural products
Empowering consumers for the green transition
